# Import

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from cvxopt import matrix, solvers
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import re
import os
from loguru import logger

import matplotlib.pyplot as plt 
%matplotlib qt

from collections import defaultdict
from datetime import datetime

import pyDRTtools as drt


Initializing pyDRTtools from c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages
['c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\python311.zip', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\DLLs', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN', '', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\win32', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\Pythonwin']
Imported basics
Imported BHT
Imported cli
Imported GUI
Imported HMC
Imported layout
Imported nearest_PD
Imported parameter_selection
Imported peak_analysis
Imported runs
Contents of pyDRTtools package: ['basics', 'GUI', 'layout', 'parameter_selection', 'peak_analysis', 'HMC', 'runs', 'nearest_PD']


In [2]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [3]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,3]
        _Zimag  = _data[:,4]
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

In [4]:
def EIS_recal_ver02(data, _phz_0 = None):
    f_poi = data[0,:]
    # Z_poi = data[1,:] * np.exp(1j*np.deg2rad(data[2,:]))
    Z_poi = data[1,:] + 1j*data[2,:]
    Y_poi = 1/Z_poi

    Rg0 = 1.611e13
    Cp0 = 1.4e-9
    
    _Rg0_rescale = 1/Rg0*np.power(f_poi,1.583)
    _Cp0_rescale = Cp0*np.power(f_poi,0.911)
    Y_org = Y_poi - _Rg0_rescale + 1j*_Cp0_rescale
    # Y_org = Y_poi - _Rg0_rescale 
    # Y_org = Y_poi + 1j*_Cp0_rescale
    # Y_org = Y_poi
    Z_org = 1/Y_org

    # Phz Calibration
    if _phz_0 is None:
        _phz_0 = np.loadtxt("./phz_Calib.txt")
    
    Z_ampC = np.abs(Z_org)
    # Z_phzC = np.angle(Z_org) - _phz_0
    Z_phzC = np.angle(Z_org) - _phz_0

    Z_rec = Z_ampC * np.exp(1j*Z_phzC)

    # C = 5e-10
    Rs0 = 100
    Z_rec = Z_rec - Rs0



    Cp0 = 5e-10
    _Cp0_rescale = Cp0 * f_poi
    Z_rec = 1/(1/Z_rec - 1j * _Cp0_rescale)

    

    # Ls0 = 1.7e-4
    Ls0 = 5e-4
    _Ls0_rescale = Ls0 * f_poi
    Z_rec = Z_rec - 1j * _Ls0_rescale

    # C = 5e-10
    Rs0 = 566
    Z_rec = Z_rec - Rs0
    
    return np.stack([f_poi, np.real(Z_rec), np.imag(Z_rec)], axis=1).T
    

# DRT Pipeline - Full Draft

In [20]:
def DRT(ch_eis, RLC_Flag=[True, True, True], custom_lambda = None):
    '''==================================================
        DRT Calculation for EIS Data
        Parameter: 
            ch_eis: 3 x n_freq Matrix: [freq, Real, Imag]
        Returen:
            tau_vec: time domain vector
            x: DRT result
            n_extend: number of extend RLC parameters
        ==================================================
    '''
    ## Freq domain data prepare
    n_freq = ch_eis.shape[1]
    freq_vec = np.flip(ch_eis[0,:])     # Flip for growing tau = 1/freq
    Z_exp = np.flip(ch_eis[1,:] + 1j*ch_eis[2,:])
    '''Hyper Parameters'''
    # Time domain parameters
    tau_min = 1/freq_vec[0]
    tau_max = 1/freq_vec[-1]
    n_tau = n_freq
    tau_vec = 1/(2*np.pi*freq_vec)
    # tau_vec = 1/(2*np.pi*100*freq_vec)
    # tau_vec = np.logspace(-10, 2, n_tau, endpoint=True)

    # log_tau = np.log(tau_vec)
    # tau_vec = np.logspace(-6, 0, n_tau, endpoint=True)
    # freq_vec = np.flip(np.logspace(0, 6, n_freq, endpoint=True))
    


    # Discretization matrices Parameters
    # Use RBF Kernel to initialize the A matrix
    RBF_shape_control = 'FWHM Coefficient' 
    RBF_coeff = 0.5
    # RBF_type = 'Piecewise Linear'
    RBF_type = 'Gaussian'
    # RBF_type = 'C0 Matern'
    # RBF_type = 'C2 Matern'
    # RBF_type = 'C4 Matern'
    # RBF_type = 'C6 Matern'
    # RBF_type = 'Inverse Quadratic'

    # Cross-validation Method for optimize lambda (Tikhonov regularization parameter) 
    # cv_type = 'GCV'     # Generalized Cross Validation
    # cv_type = 'mGCV'    # Modified Generalized Cross Validation
    cv_type = 'rGCV'    # Robust Generalized Cross Validation
    # cv_type = 'LC'      # L-curve
    # cv_type = 're-im'   # Real-Imaginary discrepancy
    # cv_type = 'kf'      # k-fold cross-validation

    '''Compute the RBF Shape Parameter Epsilon'''
    epsilon = drt.basics.compute_epsilon(freq_vec, RBF_coeff, RBF_type, RBF_shape_control)

    # logger.info(f"{epsilon}")
    # return
    '''Compute the discretization matrices'''
    A_re = drt.basics.assemble_A_re(freq_vec, tau_vec, epsilon, RBF_type)
    n_extend = np.sum(RLC_Flag)   
    if RLC_Flag[2]:
        A_re_C_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_C_0, A_re)) 
    if RLC_Flag[1]:
        A_re_L_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_L_0, A_re))
    if RLC_Flag[0]:
        A_re_R_inf  = np.ones((n_freq, 1))
        A_re        = np.hstack((A_re_R_inf, A_re))  

    A_im = drt.basics.assemble_A_im(freq_vec, tau_vec, epsilon, RBF_type)
    if RLC_Flag[2]:
        A_im_C_0    = -1/(2*np.pi*freq_vec.reshape(-1,1))
        A_im        = np.hstack((A_im_C_0, A_im))
    if RLC_Flag[1]:
        A_im_L_0    = 2*np.pi*freq_vec.reshape(-1,1)
        A_im        = np.hstack((A_im_L_0, A_im))
    if RLC_Flag[0]:
        A_im_R_inf  = np.zeros((n_freq, 1)) 
        A_im        = np.hstack((A_im_R_inf, A_im))

    A = np.vstack((A_re, A_im))


    '''Compute the differentiation matrices for Tiknonov regularization'''
    M2 = np.zeros((n_tau+n_extend, n_tau+n_extend))
    M2[n_extend:,n_extend:] = drt.basics.assemble_M_2(tau_vec, epsilon, RBF_type)

    '''Optimize lambda'''
    if custom_lambda is None:
        log_lambda_init = -7 # ln(lambda_init = 0.001)
        lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", RLC_Flag[1], log_lambda_init, cv_type)
    else: 
        lambda_opt = custom_lambda
    logger.info(f"Lambda: {lambda_opt}")
    '''Deconvolve The DRT from the EIS Data'''
    # Set Bound Constraints
    # lb = np.zeros([n_tau+n_extend])
    # bound_mat = np.eye(lb.shape[0])
    H_combined, c_combined = drt.basics.quad_format_combined(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, lambda_opt)
    G = matrix(-np.identity(Z_exp.shape[0]+n_extend))
    h = matrix(np.zeros(Z_exp.shape[0]+n_extend))
    sol = solvers.qp(matrix(H_combined), matrix(c_combined), G, h)
    

    # logger.info(f"H:{np.linalg.matrix_rank(H_combined)}, G:{np.linalg.matrix_rank(G)}")
    # logger.info(f"H_combined: {H_combined},c_combined: {c_combined}")
    # Deconvolved DRT
    x = np.array(sol['x'])
    # R_inf_DRT, L_0_DRT, C_0_DRT, DRT = x[0], x[1], x[n_extend:]
    
    return [tau_vec, x, n_extend, lambda_opt]




## Data Loading

In [21]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

# rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
# ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

# rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
# ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/06017758_归档"
# ch_id = 96     # Perfect of Perfect

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run With Error

# freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)
EISDict = gatherCSV(rootPath)
chData = readChannel(ch_id, EISDict)
freq_list = np.linspace(1000,np.shape(chData)[2]-1,101,dtype=int, endpoint=True)

if True:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData)[0]):
        ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        chData[i,:,:] = ch_eis
chData = chData[:,:,freq_list]


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:/Baihm/EISNN/Dataset/05087163_归档'

In [ ]:
if True:
    fig, axis = plt.subplots(1,4,figsize=(15,6))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(np.shape(chData)[0]):
    # for i in [0,4,2,11]:
        ch_eis = chData[i,:,:]
        # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
        _color = cmap(i/np.shape(chData)[0])
        axis[0].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[1].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
        # axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    
        # _poi_Z = np.log(np.abs(ch_eis[1,:]+1j*ch_eis[2,:]))
        # _poi_P = np.angle(ch_eis[1,:]+1j*ch_eis[2,:])
        # _poi_eis = _poi_Z * np.exp(1j*_poi_P)
        # axis[3].plot(np.real(_poi_eis), -np.imag(_poi_eis), color = _color, linewidth=2, label=f"Session {i}")
        _poi_Z = np.log(ch_eis[1,:]+1j*ch_eis[2,:])
        axis[3].plot(np.real(_poi_Z), -np.imag(_poi_Z), color = _color, linewidth=2, label=f"Session {i}")
        

axis[0].legend(frameon=False, loc='upper left')

In [ ]:
if False:
    chData = readChannel(ch_id, EISDict)
    freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int, endpoint=True)
    
    # for i in range(np.shape(chData)[0]):
    for i in range(1):
        _save_path = "D:/Baihm/EISNN/Dataset/pyDRTExample/09290511_ch013_01"
        _poi_data = chData[i,:,freq_list]
        # _poi_data = EIS_recal(chData[i,:,:])[:,freq_list].T
        _poi_data[:,0] = np.flip(_poi_data[:,0])
        _poi_data[:,1] = np.flip(_poi_data[:,1])
        _poi_data[:,2] = np.flip(_poi_data[:,2])
        # print(np.shape(np.logspace(0,6,101,endpoint=True).T))
        # _poi_data[:,0] = np.logspace(0,6,101,endpoint=True)
        with open(f"{_save_path}/Day_{i:02d}.csv", 'w') as f:
            np.savetxt(f, _poi_data, delimiter=',', fmt='%.6f',comments='')
    

In [ ]:
# %skip

# tau_vec, x_DRT = DRT(chData[0,:,freq_list].T)
# np.shape(chData[0,:,freq_list].T)

fig, axis = plt.subplots(1,3,figsize=(15,6))
cmap = plt.get_cmap('RdYlBu')
# cmap = plt.get_cmap('rainbow_r')
RLC_flag = [False, False, False]
custom_lambda = None
# custom_lambda = 1e-4
for i in range(np.shape(chData)[0]):
# for i in [4,5,6]:
    if True:
        RLC_flag = [False, False, False]
        ch_eis = chData[i,:,:]
        # df = pd.read_csv('D:/Baihm/EISNN/Download/pyDRTtools/tutorial/data/1ZARC.csv')
        # ch_eis = np.array([df['Freq'].values, df['Real'].values, df['Imag'].values])

        tau_vec, x_DRT, n_extend, _ = DRT(ch_eis, RLC_flag,custom_lambda)
        
        
        _color = cmap(i/np.shape(chData)[0])
        # if i == 5: _color = 'black'
        axis[0].semilogx(tau_vec[:], x_DRT[n_extend:], color = _color, linewidth=2, label=f"Session {i}")
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].semilogx(ch_eis[0,:], np.angle(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        
        axis[0].legend(frameon=False, loc='upper left')

    
    if False:
        RLC_flag = [True, True, False]
        ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
        # df = pd.read_csv('D:/Baihm/EISNN/Download/pyDRTtools/tutorial/data/1ZARC.csv')
        # ch_eis = np.array([df['Freq'].values, df['Real'].values, df['Imag'].values])

        tau_vec, x_DRT, n_extend, _ = DRT(ch_eis, RLC_flag,custom_lambda)
        
        
        _color = cmap(i/np.shape(chData)[0])
        axis[0].semilogx(tau_vec[:], x_DRT[n_extend:], color = _color, linewidth=2, label=f"Session {i}")
        axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].semilogx(ch_eis[0,:], np.angle(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        
        axis[0].legend(frameon=False, loc='upper left')

    # plt.plot(np.log10(ch_eis[0,:]), np.log10(np.abs(ch_eis[1,:]+1j*ch_eis[2,:])), 'r')
    # plt.plot(np.log10(ch_eis_rec[0,:]), np.log10(np.abs(ch_eis_rec[1,:]+1j*ch_eis_rec[2,:])), 'b')
    # plt.plot(np.log10(ch_day[0,:]), np.rad2deg(np.angle(ch_day[1,:]+1j*ch_day[2,:])), 'r')
    # plt.plot(np.log10(ch_day_rec[0,:]), np.rad2deg(np.angle(ch_day_rec[1,:]+1j*ch_day_rec[2,:])), 'b')


# plt.show()

c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\pyDRTtools\basics.py:59: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  out_val = integrate.quad(integrand_g_i, -50, 50, epsabs=1E-9, epsrel=1E-9)
2025-04-03 16:01:11.185 | INFO     | __main__:DRT:93 - Lambda: [0.00091188]


Optimization terminated successfully    (Exit mode 0)
            Current function value: 100494251.972294
            Iterations: 5
            Function evaluations: 2
            Gradient evaluations: 1
rGCV
     pcost       dcost       gap    pres   dres
 0: -6.8671e+11 -7.2559e+11  2e+11  1e+05  7e-02
 1: -7.0075e+11 -7.1688e+11  2e+10  8e+03  5e-03
 2: -7.0170e+11 -7.0422e+11  3e+09  8e+02  5e-04
 3: -7.0261e+11 -7.0314e+11  5e+08  9e-11  4e-16
 4: -7.0287e+11 -7.0292e+11  6e+07  6e-11  3e-16
 5: -7.0290e+11 -7.0290e+11  5e+06  6e-11  3e-16
 6: -7.0290e+11 -7.0290e+11  5e+05  9e-11  4e-16
Optimal solution found.


2025-04-03 16:01:11.563 | INFO     | __main__:DRT:93 - Lambda: [1.e-07]


Optimization terminated successfully    (Exit mode 0)
            Current function value: 10030359.267748075
            Iterations: 2
            Function evaluations: 4
            Gradient evaluations: 2
rGCV
     pcost       dcost       gap    pres   dres
 0: -5.7233e+11 -6.0949e+11  1e+11  1e+05  7e-02
 1: -6.1379e+11 -6.5986e+11  9e+10  3e+04  2e-02
 2: -6.2105e+11 -6.3216e+11  1e+10  1e+03  7e-04
 3: -6.2439e+11 -6.2683e+11  2e+09  1e+02  9e-05
 4: -6.2536e+11 -6.2620e+11  8e+08  5e-10  3e-16
 5: -6.2572e+11 -6.2581e+11  9e+07  1e-11  4e-16
 6: -6.2576e+11 -6.2578e+11  2e+07  2e-10  3e-16
 7: -6.2577e+11 -6.2578e+11  7e+06  2e-10  3e-16
 8: -6.2578e+11 -6.2578e+11  1e+06  2e-10  3e-16
 9: -6.2578e+11 -6.2578e+11  1e+05  2e-10  3e-16
Optimal solution found.


2025-04-03 16:01:11.940 | INFO     | __main__:DRT:93 - Lambda: [0.00091188]


...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 37795127.81895417
            Iterations: 5
            Function evaluations: 2
            Gradient evaluations: 1
rGCV
     pcost       dcost       gap    pres   dres
 0: -2.0608e+11 -2.1816e+11  6e+10  7e+04  7e-02
 1: -2.1039e+11 -2.1611e+11  8e+09  4e+03  4e-03
 2: -2.1089e+11 -2.1159e+11  7e+08  2e+02  2e-04
 3: -2.1108e+11 -2.1121e+11  1e+08  3e-11  4e-16
 4: -2.1111e+11 -2.1114e+11  2e+07  3e-11  4e-16
 5: -2.1112e+11 -2.1113e+11  4e+06  6e-11  4e-16
 6: -2.1113e+11 -2.1113e+11  2e+05  4e-11  3e-16
Optimal solution found.


2025-04-03 16:01:16.423 | INFO     | __main__:DRT:93 - Lambda: [0.00091188]


Optimization terminated successfully    (Exit mode 0)
            Current function value: 42247780.61971105
            Iterations: 5
            Function evaluations: 2
            Gradient evaluations: 1
rGCV
     pcost       dcost       gap    pres   dres
 0: -1.9631e+11 -2.0776e+11  6e+10  8e+04  8e-02
 1: -2.0064e+11 -2.0618e+11  8e+09  5e+03  5e-03
 2: -2.0110e+11 -2.0179e+11  7e+08  5e+01  6e-05
 3: -2.0127e+11 -2.0137e+11  9e+07  2e+00  2e-06
 4: -2.0130e+11 -2.0132e+11  2e+07  4e-01  4e-07
 5: -2.0131e+11 -2.0131e+11  3e+06  1e-02  1e-08
 6: -2.0131e+11 -2.0131e+11  2e+05  3e-04  3e-10
 7: -2.0131e+11 -2.0131e+11  7e+03  9e-06  9e-12
 8: -2.0131e+11 -2.0131e+11  2e+02  1e-07  1e-13
 9: -2.0131e+11 -2.0131e+11  2e+00  1e-09  1e-15
Optimal solution found.


2025-04-03 16:01:16.811 | INFO     | __main__:DRT:93 - Lambda: [1.e-07]


Optimization terminated successfully    (Exit mode 0)
            Current function value: 4133857.414140644
            Iterations: 2
            Function evaluations: 4
            Gradient evaluations: 2
rGCV
     pcost       dcost       gap    pres   dres
 0: -1.6359e+11 -1.7541e+11  5e+10  6e+04  9e-02
 1: -1.7862e+11 -1.9350e+11  3e+10  2e+04  2e-02
 2: -1.8117e+11 -1.8511e+11  4e+09  6e+02  8e-04
 3: -1.8194e+11 -1.8233e+11  4e+08  4e+01  5e-05
 4: -1.8208e+11 -1.8217e+11  9e+07  4e+00  6e-06
 5: -1.8210e+11 -1.8216e+11  6e+07  2e+00  2e-06
 6: -1.8213e+11 -1.8214e+11  1e+07  3e-01  5e-07
 7: -1.8213e+11 -1.8213e+11  4e+06  5e-02  7e-08
 8: -1.8213e+11 -1.8213e+11  8e+05  7e-03  1e-08
 9: -1.8213e+11 -1.8213e+11  4e+05  2e-03  2e-09
10: -1.8213e+11 -1.8213e+11  6e+04  2e-04  3e-10
11: -1.8213e+11 -1.8213e+11  1e+04  1e-10  3e-16
Optimal solution found.


# DRT Pipeline - Batch Run

## Definition

In [5]:
def DRTPrepare(ch_eis, _log_tau_min=-8, _log_tau_max=0):
        freq_vec = ch_eis[0,:]
        Z_real = ch_eis[1,:]
        Z_imag = ch_eis[2,:]
        if np.mean(np.diff(np.log(freq_vec))) > 0:
            freq_vec = np.flip(freq_vec)
            Z_real = np.flip(Z_real)
            Z_imag = np.flip(Z_imag)
        _tau_ext_scale = 0
        # _log_tau_min = np.log10(1/(2*np.pi*freq_vec[0])) - _tau_ext_scale
        # _log_tau_max = np.log10(1/(2*np.pi*freq_vec[-1])) + _tau_ext_scale
        # _log_tau_min = np.log10(1/(freq_vec[0]))  - _tau_ext_scale
        # _log_tau_max = np.log10(1/(freq_vec[-1])) + _tau_ext_scale
        # tau_vec = 1/(2*np.pi*freq_vec)
        # tau_vec = 1/freq_vec
        tau_vec = np.logspace(_log_tau_min,_log_tau_max,np.shape(freq_vec)[0],endpoint=True)
        # tau_vec = np.logspace(np.floor(_log_tau_min),np.ceil(_log_tau_max),np.shape(freq_vec)[0],endpoint=True)
        return [freq_vec, tau_vec, Z_real, Z_imag]

def DRTAssemble(freq_vec, tau_vec, RLC_Flag = [False, False, False], RBF_type = 'Gaussian'):
    '''==================================================
        Assemble A & M for Tikhonov Regularization
        Parameter: 
            freq_vec: frequency vector
            tau_vec: time domain vector
            RLC_Flag: [R_inf, L_0, C_0]
            RBF_type: RBF Kernel Type:
                1. 'Piecewise Linear'
                2. 'Gaussian'            (default)
                3. 'C0 Matern'
                4. 'C2 Matern'
                5. 'C4 Matern'
                6. 'C6 Matern'
                7. 'Inverse Quadratic'
        Returen:
            A_re: Discretization Matrix for Real Part
            A_im: Discretization Matrix for Imaginary Part
            M2: Differentiation Matrixs
        ==================================================
    '''
    ## Freq domain data prepare
    n_freq  = freq_vec.shape[0]
    n_tau   = tau_vec.shape[0]

    # Validate freq in descending order & tau in ascending order
    if np.mean(np.diff(np.log(freq_vec))) > 0:
        logger.warning("Frequency is not in descending order")
        return None
    if np.mean(np.diff(np.log(tau_vec))) < 0:
        logger.warning("Relaxation time is not in ascending order")
        return None
    # Discretization matrices Parameters
    # Use RBF Kernel to initialize the A matrix
    RBF_shape_control = 'FWHM Coefficient' 
    RBF_coeff = 0.5

    '''Compute the RBF Shape Parameter Epsilon'''
    epsilon = drt.basics.compute_epsilon(freq_vec, RBF_coeff, RBF_type, RBF_shape_control)
    # logger.info(f"epsilon: {epsilon}")

    '''Compute the discretization matrices'''
    A_re = drt.basics.assemble_A_re(freq_vec, tau_vec, epsilon, RBF_type)
    n_extend = np.sum(RLC_Flag)   
    if RLC_Flag[2]:
        A_re_C_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_C_0, A_re)) 
    if RLC_Flag[1]:
        A_re_L_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_L_0, A_re))
    if RLC_Flag[0]:
        A_re_R_inf  = np.ones((n_freq, 1))
        A_re        = np.hstack((A_re_R_inf, A_re))  

    A_im = drt.basics.assemble_A_im(freq_vec, tau_vec, epsilon, RBF_type)
    if RLC_Flag[2]:
        A_im_C_0    = -1/(2*np.pi*freq_vec.reshape(-1,1))
        A_im        = np.hstack((A_im_C_0, A_im))
    if RLC_Flag[1]:
        A_im_L_0    = 2*np.pi*freq_vec.reshape(-1,1)
        A_im        = np.hstack((A_im_L_0, A_im))
    if RLC_Flag[0]:
        A_im_R_inf  = np.zeros((n_freq, 1)) 
        A_im        = np.hstack((A_im_R_inf, A_im))

    # A = np.vstack((A_re, A_im))


    '''Compute the differentiation matrices for Tiknonov regularization'''
    M2 = np.zeros((n_tau+n_extend, n_tau+n_extend))
    M2[n_extend:,n_extend:] = drt.basics.assemble_M_2(tau_vec, epsilon, RBF_type)

    return [A_re, A_im, M2, n_extend]

def DRTDeconvolve(Z_real, Z_imag, A_re, A_im, M2, n_extend, RLC_Flag = [False, False, False], custom_lambda = None, cv_type='mGCV'):
    '''==================================================
        Deconvolve DRT from EIS Data
        Parameter: 
            freq_vec: frequency vector
            tau_vec: time domain vector
            Z_real: real part of impedance
            Z_imag: imaginary part of impedance
            custom_lambda: Tikhonov regularization parameter
            cv_type: Cross-validation Method for optimize lambda
                1. 'GCV'     
                2. 'mGCV'    (default)
                3. 'rGCV'    
                4. 'LC'      
                5. 're-im'   
                6. 'kf'      
        Returen:
            x: DRT result
            n_extend: number of extend RLC parameters
            lambda_opt: optimized lambda
            x_extend: extend RLC parameters
        ==================================================
    '''

    '''Optimize lambda'''
    if custom_lambda is None:
        log_lambda_init = -7 # ln(lambda_init = 0.001)
        lambda_opt = drt.basics.optimal_lambda(A_re, A_im, Z_real, Z_imag, M2, "Combined Re-Im Data", RLC_Flag[1], log_lambda_init, cv_type)
    else: 
        lambda_opt = custom_lambda
    # logger.info(f"lambda_opt: {epsilon}")
    
    '''Deconvolve The DRT from the EIS Data'''
    H_combined, c_combined = drt.basics.quad_format_combined(A_re, A_im, Z_real, Z_imag, M2, lambda_opt)
    G = matrix(-np.identity(Z_real.shape[0]+n_extend))
    h = matrix(np.zeros(Z_real.shape[0]+n_extend))
    sol = solvers.qp(matrix(H_combined), matrix(c_combined), G, h)
    
    # Deconvolved DRT
    x = np.array(sol['x'])
    
    if n_extend > 0:
        x_DRT = x[n_extend:]
        x_extend = x[:n_extend]
    else: 
        x_DRT = x
        x_extend = None
    return [x_DRT, lambda_opt, x_extend]



## DRT_Single

In [ ]:
def DRT_Single(ch_eis, RLC_Flag=[True, True, True], custom_lambda = 0.01):
    freq_vec, tau_vec, Z_real, Z_imag = DRTPrepare(ch_eis)
    A_re, A_im, M2, n_extend = DRTAssemble(freq_vec, tau_vec, RLC_Flag)
    x_DRT, lambda_opt, _ = DRTDeconvolve(Z_real, Z_imag, A_re, A_im, M2, n_extend, RLC_Flag, custom_lambda)

    fig, axis = plt.subplots(1,3,figsize=(15,6))
    axis[0].semilogx(tau_vec, x_DRT, linewidth=2, label=f"Session {0}")
    axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), linewidth=2, label=f"Session {0}")
    axis[2].semilogx(ch_eis[0,:], np.angle(ch_eis[1,:]+1j*ch_eis[2,:]), linewidth=2, label=f"Session {0}")
    
    # fig.show()

    # logger.info(f"Lambda: {lambda_opt}")    
    # logger.info(f"Freq: {freq_vec[0]:.2e} - {freq_vec[-1]:.2e}")
    # logger.info(f"Tau: {tau_vec[0]:.2e} - {tau_vec[-1]:.2e}")

df = pd.read_csv('D:/Baihm/EISNN/Download/pyDRTtools/tutorial/data/1ZARC.csv')
ch_eis = np.array([df['Freq'].values, df['Real'].values, df['Imag'].values])
DRT_Single(ch_eis)


     pcost       dcost       gap    pres   dres
 0: -8.4515e+04 -8.4968e+04  1e+03  1e+01  8e-08
 1: -8.4520e+04 -8.4632e+04  1e+02  4e-01  3e-09
 2: -8.4528e+04 -8.4548e+04  2e+01  5e-02  4e-10
 3: -8.4532e+04 -8.4538e+04  6e+00  5e-04  3e-12
 4: -8.4533e+04 -8.4534e+04  8e-01  2e-05  2e-13
 5: -8.4534e+04 -8.4534e+04  1e-01  1e-06  8e-15
 6: -8.4534e+04 -8.4534e+04  1e-02  5e-08  5e-16
Optimal solution found.


## DRT_Batch

In [6]:
def DRT_Batch(ch_id, EISDict, freq_list, _log_tau_min=-8, _log_tau_max=0):
    chData = readChannel(ch_id, EISDict)
    if True:
        phz_calibration = np.loadtxt("./phz_Calib.txt")
        for i in range(np.shape(chData)[0]):
            ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
            chData[i,:,:] = ch_eis


    if np.shape(chData)[0] < 3:
        logger.warning(f"Channel {ch_id} has less than 3 samples")
        return None
    # Parameters
    RLC_Flag=[True, True, True]
    custom_lambda = 1e-5

    # DRT A & M Matrix Assemble
    freq_vec, tau_vec, _, _ = DRTPrepare(chData[0,:,freq_list].T, _log_tau_min, _log_tau_max)
    A_re, A_im, M2, n_extend = DRTAssemble(freq_vec, tau_vec, RLC_Flag)
    
    # DRT Main
    ch_DRT = []
    for i in range(np.shape(chData)[0]):
    # for i in [0,1,2,3,4,5,6,7,8]:
        ch_eis = chData[i,:,freq_list].T
        # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
        freq_vec, tau_vec, Z_real, Z_imag = DRTPrepare(ch_eis)
        x_DRT, lambda_opt, x_extend = DRTDeconvolve(Z_real, Z_imag, A_re, A_im, M2, n_extend, RLC_Flag, custom_lambda)
        ch_DRT.append(x_DRT.flatten())
    return [np.array(ch_DRT), tau_vec]



### Load Data

In [13]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # Normal? 
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

# rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
# ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 outlier

## Invivo
rootPath = "D:/Baihm/EISNN/Invivo/S6006_Ver02"
ch_id = 7
EISDict = gatherCSV(rootPath)


2025-05-16 23:35:44.443 | INFO     | __main__:gatherCSV:32 - Session Begin: S6006_20241120_01
2025-05-16 23:35:44.444 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-16 23:35:44.445 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-16 23:35:44.446 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-16 23:35:44.447 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-16 23:35:44.448 | INFO     | __main__:gatherCSV:32 - Session Begin: S6006_20241127_01
2025-05-16 23:35:44.449 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-16 23:35:44.449 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-16 23:35:44.450 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-16 23:35:44.451 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-16 23:35:44.451 | INFO     | __main__:gatherCSV:32 - Session Begin: S6006_20241204_01
2025-05-16 23:35:44.452 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-16 23:35:44.452 | INFO     | __main

In [ ]:
# freq_list = np.linspace(0,5000,100,dtype=int, endpoint=False)
freq_list = np.linspace(5000-1,0,100,dtype=int, endpoint=False)
freq_list = freq_list[::-1]
freq_list

array([  49,   99,  149,  199,  249,  299,  349,  399,  449,  499,  549,
        599,  649,  699,  749,  799,  849,  899,  949,  999, 1049, 1099,
       1149, 1199, 1249, 1299, 1349, 1399, 1449, 1499, 1549, 1599, 1649,
       1699, 1749, 1799, 1849, 1899, 1949, 1999, 2049, 2099, 2149, 2199,
       2249, 2299, 2349, 2399, 2449, 2499, 2549, 2599, 2649, 2699, 2749,
       2799, 2849, 2899, 2949, 2999, 3049, 3099, 3149, 3199, 3249, 3299,
       3349, 3399, 3449, 3499, 3549, 3599, 3649, 3699, 3749, 3799, 3849,
       3899, 3949, 3999, 4049, 4099, 4149, 4199, 4249, 4299, 4349, 4399,
       4449, 4499, 4549, 4599, 4649, 4699, 4749, 4799, 4849, 4899, 4949,
       4999])

In [24]:
# freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)

# freq_list = np.linspace(0,5000,100,dtype=int, endpoint=False)

ch_DRT, tau_vec = DRT_Batch(ch_id, EISDict, freq_list,_log_tau_min=-6, _log_tau_max=0)
ch_DRT_cum = np.cumsum(ch_DRT,axis=1)


     pcost       dcost       gap    pres   dres
 0: -9.3880e+14 -1.0120e+15  1e+14  2e+06  5e-05
 1: -9.8481e+14 -1.0537e+15  1e+14  1e+06  3e-05
 2: -9.9671e+14 -1.0131e+15  2e+13  1e+04  3e-07
 3: -9.9772e+14 -9.9995e+14  2e+12  1e+03  3e-08
 4: -9.9804e+14 -9.9852e+14  5e+11  1e+01  3e-10
 5: -9.9811e+14 -9.9818e+14  8e+10  1e+00  3e-11
 6: -9.9812e+14 -9.9814e+14  1e+10  1e-02  3e-13
 7: -9.9813e+14 -9.9813e+14  4e+09  3e-03  7e-14
 8: -9.9812e+14 -9.9813e+14  8e+09  8e-09  3e-16
 9: -9.9813e+14 -9.9813e+14  1e+09  8e-09  3e-16
10: -9.9813e+14 -9.9813e+14  2e+08  1e-08  8e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.2643e+14 -9.9385e+14  1e+14  2e+06  5e-05
 1: -9.6633e+14 -1.0289e+15  1e+14  1e+06  3e-05
 2: -9.7838e+14 -9.9167e+14  1e+13  1e+04  3e-07
 3: -9.7942e+14 -9.8122e+14  2e+12  1e+03  3e-08
 4: -9.7979e+14 -9.8026e+14  5e+11  2e+01  6e-10
 5: -9.7987e+14 -9.7995e+14  8e+10  3e+00  7e-11
 6: -9.7989e+14 -9.7991e+14  1e+10  3e-02  8e-1

### 2D Plot

In [25]:
chData = readChannel(ch_id, EISDict)
fig, axis = plt.subplots(1,5,figsize=(15,6))
cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(np.shape(ch_DRT)[0]):
# for i in [0,1,2]:
    ch_eis = chData[i,:,freq_list].T
    # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
    _color = cmap(i/np.shape(chData)[0])
    axis[0].semilogx(tau_vec, np.power(ch_DRT[i],1), color = _color, linewidth=2, label=f"Session {i}", alpha = 0.5)
    axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
    axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
    axis[3].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    # axis[4].semilogx(tau_vec, ch_DRT_cum[i], color = _color, linewidth=2, label=f"Session {i}")
    # axis[3].plot(chData[i,1,1000:], -chData[i,2,1000:], color = _color, linewidth=2, label=f"Session {i}")
 
 
# axis[0].legend(frameon=False, loc='upper left')

In [116]:
chData = readChannel(ch_id, EISDict)
fig, axis = plt.subplots(2,1,figsize=(15,6))
cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(np.shape(ch_DRT)[0]):
# for i in [0,-1]:
    ch_eis = chData[i,:,freq_list].T
    # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
    _color = cmap(i/np.shape(chData)[0])
    axis[0].semilogx(tau_vec, np.power(ch_DRT[i],1), color = _color, linewidth=2, label=f"Session {i}", alpha = 0.5)
    axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
    # axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
    # axis[3].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    # axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    # axis[4].semilogx(tau_vec, ch_DRT_cum[i], color = _color, linewidth=2, label=f"Session {i}")
    # axis[3].plot(chData[i,1,1000:], -chData[i,2,1000:], color = _color, linewidth=2, label=f"Session {i}")
 
 
# axis[0].legend(frameon=False, loc='upper left')

In [ ]:
# chData = readChannel(ch_id, EISDict)
# fig, axis = plt.subplots(1,1,figsize=(15,6))
# cmap = plt.get_cmap('rainbow_r')
# for i in range(np.shape(ch_DRT)[0]):
# # for i in [0,1,2,3,4,5,6,9,10,11,12]:
#     ch_eis = chData[i,:,freq_list].T
#     # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
#     _color = cmap(i/np.shape(chData)[0])
#     axis.loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
    
 
# # axis[0].legend(frameon=False, loc='upper left')

### 3D Plot

In [ ]:
# # 3D Plot
# fig = plt.figure()
# ax0 = fig.add_subplot(121, projection='3d')
# ax1 = fig.add_subplot(122, projection='3d')
# init_elev = 21  # 仰角
# init_azim = 55  # 方位角
# ax0.view_init(elev=init_elev, azim=init_azim)
# ax1.view_init(elev=init_elev, azim=init_azim)

# # x = np.log10(tau_vec).flatten()
# x = np.array(range(ch_DRT.shape[0]))
# y = np.array(range(ch_DRT.shape[1]))
# X,Y = np.meshgrid(x,y,indexing='ij')
# ax0.plot_surface(X, Y, ch_DRT[:,:ch_DRT.shape[1]], cmap='viridis_r', alpha=0.8)


# ch_Data_ds = np.abs(chData[:,1,:] + 1j*chData[:,2,:])
# x = np.array(range(ch_Data_ds.shape[0]))
# y = np.array(range(ch_Data_ds.shape[1]))
# X,Y = np.meshgrid(x,y,indexing='ij')
# ax1.plot_surface(X, Y, np.log10(ch_Data_ds), cmap='viridis_r', alpha=0.8)




### Contour Plot

In [ ]:
# x = np.array(range(ch_DRT.shape[1]))
# y = np.array(range(ch_DRT.shape[0]))

# X,Y = np.meshgrid(x,y)

# fig = plt.figure()
# gs = plt.GridSpec(1,1)
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0, hspace=0.5)
# ax = plt.subplot(gs[0,0])
# cs = ax.contourf(X,Y,ch_DRT,cmap='plasma')

## DRT_dimentional deduction

### Normal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS
import umap.umap_ as umap  # 请确保安装了 umap-learn

np.random.seed(42)
ch_EIS = np.abs(chData[:,1,freq_list] + 1j*chData[:,2,freq_list])
# data = ch_DRT
# data = np.log(ch_DRT_cum)
data = np.log(ch_EIS)

_order = 3
methods = {
    'PCA': PCA(n_components=_order),
    't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
    'Isomap': Isomap(n_components=_order),
    'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
    'MDS': MDS(n_components=_order, random_state=42),
    'UMAP': umap.UMAP(n_components=_order, random_state=42)
}

embeddings = {}
emb_dist = {}
for name, method in methods.items():
    embedding = method.fit_transform(data)
    embeddings[name] = embedding
    
    _x = embedding[:,0].flatten()
    _y = embedding[:,1].flatten()

    emb_dist[name] = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                         (_y[:, np.newaxis] - _y[np.newaxis, :])**2)



#### 2D Plot

In [ ]:

fig, axis = plt.subplots(3,4,figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:,0].flatten()
    _y = emb[:,1].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                         (_y[:, np.newaxis] - _y[np.newaxis, :])**2)
    # _dist = emb_dist[name]

    axis[np.int16(i/2),(i%2)*2].scatter(emb[:, 0], emb[:, 1], c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    axis[np.int16(i/2),(i%2)*2].set_title(name)

    s = axis[np.int16(i/2),(i%2)*2+1].imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2),(i%2)*2+1])
plt.tight_layout()
plt.show()


#### 3D Plot

In [ ]:
# fig, axis = plt.subplots(3, 4, figsize=(12, 6), subplot_kw={'projection': '3d'})  # 指定3D投影
fig = plt.figure(figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:, 0].flatten()
    _y = emb[:, 1].flatten()
    _z = emb[:, 2].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                    (_y[:, np.newaxis] - _y[np.newaxis, :])**2 + 
                    (_z[:, np.newaxis] - _z[np.newaxis, :])**2)

    ax = fig.add_subplot(3,4,i*2+1, projection='3d')

    # 3D散点图
    sc = ax.scatter(_x, _y, _z, c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    ax.set_title(name)

    # 2D距离矩阵可视化
    
    ax = fig.add_subplot(3,4,i*2+2)
    s = ax.imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2), (i%2)*2+1])

plt.show()


### Raw Data 添加顺序信息
这里需要调整顺序信息在整个DRT数据中的权重，容易因为顺序信息过强导致异常值不显著

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS
import umap.umap_ as umap  # 请确保安装了 umap-learn

np.random.seed(42)
ch_EIS = np.abs(chData[:,1,freq_list] + 1j*chData[:,2,freq_list])

index_scale = np.max(ch_DRT) * 0.5
index_vec = ((np.arange(np.shape(ch_DRT)[0])+1) * index_scale ).reshape(-1,1)
ch_DRT_ext = np.hstack((ch_DRT, index_vec))


# data = ch_DRT
# data = np.log(ch_EIS)
data =np.hstack((ch_DRT, index_vec))
# data =np.hstack((ch_EIS, index_vec))



_order = 3
methods = {
    'PCA': PCA(n_components=_order),
    't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
    'Isomap': Isomap(n_components=_order),
    'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
    'MDS': MDS(n_components=_order, random_state=42),
    'UMAP': umap.UMAP(n_components=_order, random_state=42)
}

embeddings = {}
for name, method in methods.items():
    embedding = method.fit_transform(data)
    embeddings[name] = embedding


#### 2D Plot

In [ ]:

fig, axis = plt.subplots(3,4,figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:,0].flatten()
    _y = emb[:,1].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                         (_y[:, np.newaxis] - _y[np.newaxis, :])**2)


    axis[np.int16(i/2),(i%2)*2].scatter(emb[:, 0], emb[:, 1], c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    axis[np.int16(i/2),(i%2)*2].set_title(name)

    s = axis[np.int16(i/2),(i%2)*2+1].imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2),(i%2)*2+1])
plt.tight_layout()
plt.show()


#### 3D Plot

In [ ]:

fig = plt.figure(figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:, 0].flatten()
    _y = emb[:, 1].flatten()
    _z = emb[:, 2].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                    (_y[:, np.newaxis] - _y[np.newaxis, :])**2 + 
                    (_z[:, np.newaxis] - _z[np.newaxis, :])**2)

    ax = fig.add_subplot(3,4,i*2+1, projection='3d')

    # 3D散点图
    sc = ax.scatter(_x, _y, _z, c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    ax.set_title(name)

    # 2D距离矩阵可视化
    
    ax = fig.add_subplot(3,4,i*2+2)
    s = ax.imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2), (i%2)*2+1])

plt.show()


### PCA添加顺序信息

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS
import umap.umap_ as umap  # 请确保安装了 umap-learn

np.random.seed(42)
ch_EIS = np.abs(chData[:,1,freq_list] + 1j*chData[:,2,freq_list])


data = ch_DRT
# data = np.log(ch_EIS)


_order = 3
methods = {
    'PCA': PCA(n_components=_order),
    't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
    'Isomap': Isomap(n_components=_order),
    'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
    'MDS': MDS(n_components=_order, random_state=42),
    'UMAP': umap.UMAP(n_components=_order, random_state=42)
}

embeddings = {}
for name, method in methods.items():
    embedding = method.fit_transform(data)
    embeddings[name] = embedding


#### 3D Plot

In [ ]:

fig = plt.figure(figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:, 0].flatten() / np.max(emb[:, 0])
    _y = emb[:, 1].flatten() / np.max(emb[:, 1])
    _z = (np.arange(np.shape(emb)[0]) + 1).flatten() / np.shape(emb)[0] 

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                    (_y[:, np.newaxis] - _y[np.newaxis, :])**2 + 
                    (_z[:, np.newaxis] - _z[np.newaxis, :])**2)

    ax = fig.add_subplot(3,4,i*2+1, projection='3d')

    # 3D散点图
    sc = ax.scatter(_x, _y, _z, c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    ax.set_title(name)

    # 2D距离矩阵可视化
    
    ax = fig.add_subplot(3,4,i*2+2)
    s = ax.imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2), (i%2)*2+1])

plt.show()


### EIS变化张量

In [ ]:
def assemble_dist_tensor(data_cpx):
    n_sample = np.shape(data_cpx)[1]
    dist_tensor = []
    for i in range(n_sample):
        _vec = data_cpx[:,i].reshape(-1,1)
        dist_tensor.append(np.real(_vec - _vec.T))
        dist_tensor.append(np.imag(_vec - _vec.T))
    dist_tensor = np.array(dist_tensor).T
    return dist_tensor.reshape(np.shape(dist_tensor)[0],-1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS
import umap.umap_ as umap  # 请确保安装了 umap-learn

np.random.seed(42)

ch_EIS_cpx = chData[:,1,freq_list] + 1j*chData[:,2,freq_list]
dist_tensor = assemble_dist_tensor(ch_EIS_cpx)
data = dist_tensor
# data = np.log(ch_DRT_cum)
# data = np.log(ch_EIS)

_order = 3
methods = {
    'PCA': PCA(n_components=_order),
    't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
    'Isomap': Isomap(n_components=_order),
    'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
    'MDS': MDS(n_components=_order, random_state=42),
    'UMAP': umap.UMAP(n_components=_order, random_state=42)
}

embeddings = {}
for name, method in methods.items():
    embedding = method.fit_transform(data)
    embeddings[name] = embedding



#### 2D

In [ ]:

fig, axis = plt.subplots(3,4,figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:,0].flatten()
    _y = emb[:,1].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                    (_y[:, np.newaxis] - _y[np.newaxis, :])**2)


    axis[np.int16(i/2),(i%2)*2].scatter(emb[:, 0], emb[:, 1], c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    axis[np.int16(i/2),(i%2)*2].set_title(name)

    s = axis[np.int16(i/2),(i%2)*2+1].imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2),(i%2)*2+1])
plt.tight_layout()
plt.show()


#### 3D

In [ ]:

fig = plt.figure(figsize=(12,6))
for i, (name, emb) in enumerate(embeddings.items()):
    _x = emb[:, 0].flatten()
    _y = emb[:, 1].flatten()
    _z = emb[:, 2].flatten()

    _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                    (_y[:, np.newaxis] - _y[np.newaxis, :])**2 + 
                    (_z[:, np.newaxis] - _z[np.newaxis, :])**2)

    ax = fig.add_subplot(3,4,i*2+1, projection='3d')

    # 3D散点图
    sc = ax.scatter(_x, _y, _z, c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
    ax.set_title(name)

    # 2D距离矩阵可视化
    
    ax = fig.add_subplot(3,4,i*2+2)
    s = ax.imshow(_dist, cmap='coolwarm', interpolation='nearest')
    fig.colorbar(s, ax=axis[np.int16(i/2), (i%2)*2+1])

plt.show()


## DRT or EIS based OD

### HMM
由于需要手动设置状态，无法很好的区分异常

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

# 假设你的数据是一个 16x101 的矩阵，这里用随机数据模拟
np.random.seed(42)
# data = ch_DRT
data = np.log(ch_EIS)

# 如果需要，也可以对数据做预处理，例如归一化

# 定义最大可能状态数，比如设为5
max_states = 4

# 初始化高斯隐马尔可夫模型
# covariance_type 可以根据数据特性选择 'diag' 或 'full'
model = hmm.GaussianHMM(n_components=max_states, covariance_type='diag', n_iter=100, random_state=42)

# 拟合模型
model.fit(data)

# 使用模型预测隐状态序列（返回长度为16的一维数组）
hidden_states = model.predict(data)
print("预测的隐状态序列:", hidden_states)

# 输出状态转移矩阵，观察哪些状态有较高的占用概率
print("状态转移矩阵:\n", model.transmat_)

# 可视化隐状态随时间的变化
plt.figure(figsize=(10, 4))
plt.plot(hidden_states, marker='o', linestyle='-')
plt.xlabel('Time Stamp')
plt.ylabel('Hidden State')
plt.title('HMM Hidden State Series')
plt.grid(True)
plt.show()


### Bayesian Change Point Detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import t  # 导入 Student-t 分布

# ----------------------------
# 1. 数据准备与降维
# ----------------------------
np.random.seed(42)
data = ch_DRT
# data = np.random.randn(16, 101)
# data[8:] += 5  # 模拟状态转移

pca = PCA(n_components=1)
data_1d = pca.fit_transform(data).flatten()

# ----------------------------
# 2. BOCPD实现（Univariate case）
# ----------------------------
T = len(data_1d)
R = np.zeros((T + 1, T + 1))
R[0, 0] = 1

# Normal-Inverse-Gamma先验参数
mu0 = 0.0
kappa0 = 1.0
alpha0 = 1.0
beta0 = 1.0

# 初始化足够统计量，使用字典存储，key为运行长度
mu = {0: mu0}
kappa = {0: kappa0}
alpha = {0: alpha0}
beta = {0: beta0}

def predictive_pdf(x, mu_val, kappa_val, alpha_val, beta_val):
    """
    预测分布为 Student-t 分布：
      - 自由度: nu = 2 * alpha_val
      - 位置: mu_val
      - 尺度: sqrt( beta_val*(kappa_val+1) / (alpha_val*kappa_val) )
    """
    nu = 2 * alpha_val
    scale = np.sqrt(beta_val * (kappa_val + 1) / (alpha_val * kappa_val))
    return t.pdf(x, df=nu, loc=mu_val, scale=scale)

def constant_hazard(lam):
    """常数危险函数"""
    return lambda r: np.full(r.shape, 1.0 / lam)

hazard = constant_hazard(100)
cp_probabilities = []

for time_idx in range(1, T + 1):
    x = data_1d[time_idx - 1]
    pred_probs = np.zeros(time_idx)
    for r in range(time_idx):
        pred_probs[r] = predictive_pdf(x, mu[r], kappa[r], alpha[r], beta[r])
    
    growth_probs = R[time_idx - 1, :time_idx] * (1 - hazard(np.arange(1, time_idx + 1))) * pred_probs
    cp_prob = np.sum(R[time_idx - 1, :time_idx] * hazard(np.arange(1, time_idx + 1)) * pred_probs)
    
    R[time_idx, 1:time_idx + 1] = growth_probs
    R[time_idx, 0] = cp_prob
    R[time_idx, :time_idx + 1] /= np.sum(R[time_idx, :time_idx + 1])
    
    cp_probabilities.append(R[time_idx, 0])
    
    new_mu = {}
    new_kappa = {}
    new_alpha = {}
    new_beta = {}
    
    new_mu[0] = mu0
    new_kappa[0] = kappa0
    new_alpha[0] = alpha0
    new_beta[0] = beta0
    
    for r in range(1, time_idx + 1):
        new_mu[r] = (kappa[r - 1] * mu[r - 1] + x) / (kappa[r - 1] + 1)
        new_kappa[r] = kappa[r - 1] + 1
        new_alpha[r] = alpha[r - 1] + 0.5
        new_beta[r] = beta[r - 1] + 0.5 * ((x - mu[r - 1]) ** 2 * kappa[r - 1] / (kappa[r - 1] + 1))
    
    mu = new_mu
    kappa = new_kappa
    alpha = new_alpha
    beta = new_beta

# ----------------------------
# 3. 可视化
# ----------------------------
plt.figure(figsize=(10, 6))
plt.stem(range(1, T + 1), cp_probabilities, basefmt=" ")
plt.xlabel('时间点')
plt.ylabel('变点概率')
plt.title('贝叶斯在线变点检测 - 变点概率')
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1, T + 1), data_1d, marker='o')
plt.xlabel('时间点')
plt.ylabel('PCA降维后数据')
plt.title('1D降维数据及变点检测')
plt.axvline(x=8, color='red', linestyle='--', label='模拟变点（时刻8）')
plt.legend()
plt.show()


### OD - IsoForest / LOF / SVM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
# from pyod.models.pca import PCA as PCA_OD

# 生成示例数据
# np.random.seed(42)
ch_EIS = np.abs(chData[:,1,freq_list] + 1j*chData[:,2,freq_list])
# data = ch_DRT
data = np.log(ch_EIS)

methods = {
    'PCA': PCA(n_components=_order),
    't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
    'Isomap': Isomap(n_components=_order),
    'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
    'MDS': MDS(n_components=_order, random_state=42),
    'UMAP': umap.UMAP(n_components=_order, random_state=42)
}


# 使用PCA将数据降维到2D，以便可视化
data_2d = methods['PCA'].fit_transform(data)

# 定义子图布局
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes = axes.flatten()

# 1. 使用PCA进行异常检测
# pca_od = PCA_OD(contamination=0.1)
# pca_od.fit(data)
# pca_scores = pca_od.decision_function(data)
# pca_predictions = pca_od.predict(data)
# axes[0].scatter(data_2d[:, 0], data_2d[:, 1], c=pca_predictions, cmap='coolwarm', edgecolor='k')
# axes[0].set_title('PCA Anomaly Detection')

# 2. 使用Isolation Forest进行异常检测
iso_forest = IsolationForest(contamination=0.1, random_state=100)
iso_forest.fit(data)
iso_predictions = iso_forest.predict(data)
iso_predictions = np.where(iso_predictions == 1, 0, 1)  # 转换为0表示正常，1表示异常
axes[0].scatter(data_2d[:, 0], data_2d[:, 1], c=iso_predictions, cmap='coolwarm', edgecolor='k')
axes[0].set_title('Isolation Forest Anomaly Detection')

# 3. 使用LOF进行异常检测
lof = LocalOutlierFactor(n_neighbors=4, contamination=0.2)
lof_predictions = lof.fit_predict(data)
lof_predictions = np.where(lof_predictions == 1, 0, 1)  # 转换为0表示正常，1表示异常
axes[1].scatter(data_2d[:, 0], data_2d[:, 1], c=lof_predictions, cmap='coolwarm', edgecolor='k')
axes[1].set_title('LOF Anomaly Detection')

# 4. 使用One-Class SVM进行异常检测
oc_svm = OneClassSVM(nu=0.1, kernel='rbf', gamma='auto')
oc_svm.fit(data)
oc_predictions = oc_svm.predict(data)
oc_predictions = np.where(oc_predictions == 1, 0, 1)  # 转换为0表示正常，1表示异常
axes[2].scatter(data_2d[:, 0], data_2d[:, 1], c=oc_predictions, cmap='coolwarm', edgecolor='k')
axes[2].set_title('One-Class SVM Anomaly Detection')

# 设置图形标题和布局
plt.suptitle('Anomaly Detection Results Using Different Algorithms')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


### Dist Based Cluster

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 生成模拟的 16x16 距离矩阵（对称矩阵，模拟样本间距离）
np.random.seed(42)
A = emb_dist['PCA']

# PCA 降维用于可视化
pca = PCA(n_components=2)
X_2D = pca.fit_transform(A)

# ==================== 层次聚类 ====================
Z = linkage(A, method='average',optimal_ordering=True)  # 使用 Ward 方式进行层次聚类
clusters_hierarchical = fcluster(Z, t=3, criterion='maxclust')

# ==================== DBSCAN ====================
dbscan = DBSCAN(metric='precomputed', eps=0.5, min_samples=2)
clusters_dbscan = dbscan.fit_predict(A)

# ==================== LOF ====================
lof = LocalOutlierFactor(metric='precomputed', n_neighbors=5)
lof_outlier_scores = lof.fit_predict(A)

# 可视化函数
def plot_results(title, labels):
    plt.figure(figsize=(6, 4))
    plt.scatter(X_2D[:, 0], X_2D[:, 1], c=labels, cmap='coolwarm', edgecolors='k', s=100)
    plt.colorbar()
    plt.title(title)
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.show()

# 绘制层次聚类结果
plot_results("Hierarchical Clustering", clusters_hierarchical)

# 绘制 DBSCAN 结果
plot_results("DBSCAN Clustering", clusters_dbscan)

# 绘制 LOF 结果（-1 为异常点）
plot_results("Local Outlier Factor (LOF)", lof_outlier_scores)


### Dist Based series order 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram, optimal_leaf_ordering
from scipy.spatial.distance import squareform

# 模拟 16x16 的相关性矩阵（对称，主对角线为1）
np.random.seed(42)
D = emb_dist['MDS']/np.max(emb_dist['MDS'])  # 16x16 距离矩阵
# D = (pca_dist-np.mean(pca_dist))  # 16x16 距离矩阵

# --- 方法1：修改距离矩阵 ---
lambda_penalty = 0  # 调节因子，可根据需要调整
n = D.shape[0]
indices = np.arange(n)
# 构造原始顺序差值矩阵 |i - j|
diff_matrix = np.abs(indices.reshape(-1, 1) - indices.reshape(1, -1)) / n
P = lambda_penalty * diff_matrix
D_mod = D + P  # 修改后的距离矩阵

# 转换为 condensed 距离向量供 linkage 使用
condensed_D_mod = squareform(D_mod)
# 使用平均链接法构造层次聚类树
Z_mod = linkage(condensed_D_mod, method='average',optimal_ordering=True)
# Z_mod = linkage(condensed_D_mod, method='ward',optimal_ordering=True)
# 可选：利用 optimal_leaf_ordering 获得最优叶排序
Z_mod_olo = optimal_leaf_ordering(Z_mod, condensed_D_mod)
dendro1 = dendrogram(Z_mod_olo, no_plot=True)
order1 = dendro1['leaves']
# print("方法1排序结果：", order1)

# 可视化排序后矩阵
sorted_A1 = D[np.ix_(order1, order1)]
plt.figure(figsize=(6, 5))
# sns.heatmap(sorted_A1, annot=False, cmap='coolwarm')
sns.heatmap(D+P, annot=False, cmap='coolwarm')
plt.title("Ordered Matrix Based on Modified Distance")
plt.show()

# 可视化层次聚类树
plt.figure(figsize=(8, 4))
# dendrogram(Z_mod_olo, labels=np.arange(n))
dendrogram(Z_mod, labels=np.arange(n))
plt.title("H")
plt.xlabel("Sample Index")
plt.ylabel("Dist")
plt.show()


### Constrained Optimal Leaf Ordering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import SpectralClustering
from sklearn.metrics import pairwise_distances

# 示例16x16距离矩阵
np.random.seed(42)
distance_matrix = emb_dist['PCA']/np.max(emb_dist['PCA'])  # 16x16 距离矩阵

from sklearn.mixture import GaussianMixture

# 使用MDS将距离矩阵转换为坐标形式
mds = MDS(n_components=2, dissimilarity='euclidean', random_state=42)
coords = mds.fit_transform(ch_DRT)

# 进行GMM聚类
n_components = 3  # 假设分为4个高斯成分
gmm = GaussianMixture(n_components=n_components, random_state=42)
gmm.fit(coords)
labels = gmm.predict(coords)

# 可视化GMM聚类结果
plt.figure(figsize=(8, 6))
plt.scatter(coords[:, 0], coords[:, 1], c=labels, cmap='viridis', s=100, )
plt.title("高斯混合模型 - MDS降维后的聚类结果")
plt.xlabel("MDS Dimension 1")
plt.ylabel("MDS Dimension 2")
plt.show()



# DRT_PCA

In [ ]:
# # import numpy as np
# # import matplotlib.pyplot as plt

# # 假设你的数据是 16x101 的 numpy 数组
# # freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
 
# chData = readChannel(ch_id, EISDict)
# ch_EIS = np.abs(chData[:,1,freq_list] + 1j*chData[:,2,freq_list])
# # data = np.log(ch_EIS)
# # data = ch_EIS

# # data = np.log(ch_DRT+1)
# data = ch_DRT_cum

# # 1. 数据中心化（去均值）
# mean = np.mean(data, axis=0)
# data_centered = data - mean

# # 2. 计算协方差矩阵
# cov_matrix = np.cov(data_centered, rowvar=False)

# # 3. 计算特征值和特征向量
# eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# # 4. 按特征值大小排序
# idx = np.argsort(eigenvalues)[::-1]  # 从大到小排序
# eigenvalues = eigenvalues[idx]
# eigenvectors = eigenvectors[:, idx]

# # 5. 选择前 k 个主成分（这里 k=2）
# k = 2
# top_eigenvectors = np.hstack((eigenvectors[:, 0].reshape(-1,1),eigenvectors[:, 1].reshape(-1,1)))

# # 6. 投影数据到主成分空间
# pca_data = np.dot(data_centered, top_eigenvectors)

# # 7. 计算PCA下距离
# _x = pca_data[:,0].flatten()
# _y = pca_data[:,1].flatten()

# pca_dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
#                          (_y[:, np.newaxis] - _y[np.newaxis, :])**2)

# # pca_dist_Z = (pca_dist-np.mean(pca_dist))/np.std(pca_dist)


# fig, axis = plt.subplots(1,2,figsize=(12,6))

# # 7. 画出前两个主成分
# axis[0].scatter(pca_data[:, 0], pca_data[:, 1],c=np.arange(len(pca_data)), cmap='rainbow_r')
# axis[0].set_xlabel("Principal Component 1")
# axis[0].set_ylabel("Principal Component 2")
# axis[0].set_title("PCA Projection")
# # axis[0].set_aspect('equal')



# # 绘制热图
# # s = axis[1].imshow(pca_dist_Z, vmin=-1, vmax=1, cmap='coolwarm', interpolation='nearest')
# s = axis[1].imshow(pca_dist, cmap='coolwarm', interpolation='nearest')
# # s = axis[1].imshow(pca_dist, cmap='coolwarm', interpolation='nearest')
# fig.colorbar(s, ax=axis[1])
# axis[1].set_ylabel("Point Index")
# axis[1].set_title("Distance Matrix Heatmap")





# Lambda Discussion

In [ ]:
# Discuss lambda
if False:
    chData = readChannel(ch_id, EISDict)
    freq_list = np.linspace(500,np.shape(chData)[2]-1,101,dtype=int, endpoint=True)
    # tau_vec, x_DRT = DRT(chData[0,:,freq_list].T)
    # np.shape(chData[0,:,freq_list].T)

    fig, axis = plt.subplots(1,3,figsize=(15,6))
    cmap = plt.get_cmap('RdYlBu')
    # cmap = plt.get_cmap('rainbow_r')
    RLC_flag = [False, False, False]
    # custom_lambda = None
    custom_lambda_list = np.logspace(-6, 0, 7, endpoint=True)
    # for i in range(np.shape(chData)[0]):
    for i in range(7):
        if True:
            ch_eis = chData[0,:,freq_list].T
            # df = pd.read_csv('D:/Baihm/EISNN/Download/pyDRTtools/tutorial/data/1ZARC.csv')
            # ch_eis = np.array([df['Freq'].values, df['Real'].values, df['Imag'].values])

            tau_vec, x_DRT, n_extend, _ = DRT(ch_eis, RLC_flag,custom_lambda_list[i])
            
            
            _color = cmap(i/np.shape(custom_lambda_list)[0])
            # if i == 5: _color = 'black'
            axis[0].semilogx(tau_vec[:], x_DRT[n_extend:], color = _color, linewidth=2, label=f"$\lambda$={custom_lambda_list[i]:.1e}")
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"$\lambda$={custom_lambda_list[i]:.1e}")
            axis[2].semilogx(ch_eis[0,:], np.angle(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"$\lambda$={custom_lambda_list[i]:.1e}")
            
            axis[0].legend(frameon=False, loc='upper left')

        
        if False:
            ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
            # df = pd.read_csv('D:/Baihm/EISNN/Download/pyDRTtools/tutorial/data/1ZARC.csv')
            # ch_eis = np.array([df['Freq'].values, df['Real'].values, df['Imag'].values])

            tau_vec, x_DRT, n_extend, _ = DRT(ch_eis, RLC_flag)
            
            
            _color = cmap(i/np.shape(chData)[0])
            axis[0].semilogx(tau_vec[:], x_DRT[n_extend:], color = _color, linewidth=2, label=f"Session {i}")
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
            axis[2].semilogx(ch_eis[0,:], np.angle(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
            
            axis[0].legend(frameon=False, loc='upper left')

        # plt.plot(np.log10(ch_eis[0,:]), np.log10(np.abs(ch_eis[1,:]+1j*ch_eis[2,:])), 'r')
        # plt.plot(np.log10(ch_eis_rec[0,:]), np.log10(np.abs(ch_eis_rec[1,:]+1j*ch_eis_rec[2,:])), 'b')
        # plt.plot(np.log10(ch_day[0,:]), np.rad2deg(np.angle(ch_day[1,:]+1j*ch_day[2,:])), 'r')
        # plt.plot(np.log10(ch_day_rec[0,:]), np.rad2deg(np.angle(ch_day_rec[1,:]+1j*ch_day_rec[2,:])), 'b')


    # plt.show()